In [1]:
# 99k bangla words taken from https://github.com/hermitdave/FrequencyWords/tree/master/content/2018/bn
# 2 lac bangla words taken from https://github.com/rspeer/wordfreq
# symspell github https://github.com/mammothb/symspellpy

In [2]:
!pip install symspellpy
!pip install fasttext-langdetect
!pip install wordfreq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.5/125.5 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for fasttext-langdetect: filename=fasttext_langdetect-1.0.3-py3-none-any.whl size=6548 sha256=91d1603335ef4092dfd72da08df83b463c291ee2d6836d6c4e27af16b8bbaa8c
  Stored in directory: /root/.cache/pip/wheels/7d/cb/96/0fb678c75ebffce072159f58d3895245e048e41a4741d4382c
Successfully built fasttext-langdetect
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 4.6 MB/s eta 0:00:00


In [3]:
!pip install wget

  Preparing metadata (setup.py) ... - done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=103ad0d7af1a8b9e12071d1a6f29c1952cb0806d2690dc031ba91f5076f7a05f
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [4]:
!pip install bnunicodenormalizer

  Preparing metadata (setup.py) ... - done
  Created wheel for bnunicodenormalizer: filename=bnunicodenormalizer-0.1.1-py3-none-any.whl size=21913 sha256=cb0e9d91d8e111980a5c4d88c42a94d8ad17bf1b9dc786b63e454d8603ccddc0
  Stored in directory: /root/.cache/pip/wheels/a2/4f/12/67639dde670a3ccd1edcb84d712bb73eb734f822a365109af4
Successfully built bnunicodenormalizer


In [5]:
import numpy as np
import pandas as pd
import random
import ast
from tqdm import tqdm
from IPython import display as ipd

# visualization
import matplotlib.pyplot as plt
from tabulate import tabulate
import seaborn as sns
#system files
import os
import json
import re
import glob

import zipfile
import shutil
import gc
from pydub import AudioSegment
from joblib import Parallel, delayed

from ftlangdetect import detect

from wordfreq import word_frequency ,top_n_list,get_frequency_dict,zipf_frequency
from collections import Counter

from pandarallel import pandarallel
from bnunicodenormalizer import Normalizer 
pandarallel.initialize(progress_bar=True,nb_workers=8)
tqdm.pandas()
bnorm=Normalizer()

import warnings
warnings.filterwarnings('ignore')

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [6]:
def normalize(sen):
    _words = [bnorm(word)['normalized']  for word in sen.split()]
    return " ".join([word for word in _words if word is not None]) 

In [7]:
def remove_punctuations(my_str):
    punctuations = '''````£|¢|Ñ+-*/=EROero৳০১২৩৪৫৬৭৮৯012–34567•89।!()-[]{};:'"“\’…,<>.‚/?@#$%^&*_~‘—॥”‰🤣⚽️✌�￰৷￰'''
    no_punct = ""
    for char in my_str:
        if char not in punctuations:
            no_punct = no_punct + char
    return no_punct

In [8]:
!wget https://raw.githubusercontent.com/hermitdave/FrequencyWords/master/content/2018/bn/bn_full.txt

--2022-12-01 22:58:16--  https://raw.githubusercontent.com/hermitdave/FrequencyWords/master/content/2018/bn/bn_full.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2352582 (2.2M) [text/plain]
Saving to: ‘bn_full.txt’

bn_full.txt         100%[===================>]   2.24M  --.-KB/s    in 0.02s   

2022-12-01 22:58:16 (92.5 MB/s) - ‘bn_full.txt’ saved [2352582/2352582]



In [9]:
df2 = pd.read_csv("bn_full.txt", sep=" ")

df2.reset_index(inplace=True)
df2.columns=['in','word','frequency']

df2 = df2.drop('in',axis=1)
df2['word'] = df2['word'].apply(lambda x : remove_punctuations(x))
df2["word"]=df2["word"].parallel_apply(lambda x:normalize(x))
df2['lang'] = df2['word'].apply(detect)
df2['lang_bn'] = df2['lang'].apply(lambda x:x['lang'])
df2=df2[df2.lang_bn == 'bn']
df2 = df2.drop(['lang','lang_bn'],axis=1)
#df2=df2[df2.word.str.len() >=2]
df_99k = df2
df_99k

word  frequency
0              আমি      62498
1               না      55643
2             তুমি      34867
3             আমার      33644
4               কি      27254
...            ...        ...
99986  কারাধ্রাসএর          1
99987       গিরিপথ          1
99988     ফ্রোডোকে          1
99989  কারাধ্রাসের          1
99991   অ্যাডলারকে          1

[83968 rows x 2 columns]

In [10]:
prothom = pd.read_csv('/kaggle/input/bn-corpus-prothom-alo/bangla_text_data.csv')
prothom

sentence
0        পশ্চিমবঙ্গের বরখাস্ত হওয়া মন্ত্রী পার্থ চট্টোপ...
1                        আর জামিনের আবেদন করলেন না অর্পিতা
2        আজ শুক্রবার দিনের ইডি হেফাজত শেষে তাঁদের হাজির...
3        সেখানে পার্থর আইনজীবী জামিনের আবেদন জানালেও অর...
4                               অর্পিতা মুখার্জি কারাগারেই
...                                                    ...
8418468            চারটি অভিযোগে তাঁকে মৃত্যুদণ্ড দেওয়া হয়
8418469  এর মধ্যে রাজনৈতিক গ্রুপকে গণহত্যার দায়ের বিষয়ট...
8418470                                           যা প্রথম
8418471  অপর একটি অভিযোগে তাঁকে বছরের কারাদণ্ড দিয়েছেন ...
8418472                                                NaN

[8418473 rows x 1 columns]

In [11]:
prothom = prothom.dropna(how='any')

prothom['lang'] = prothom['sentence'].apply(detect)
prothom['lang_bn'] = prothom['lang'].apply(lambda x:x['lang'])
prothom=prothom[prothom.lang_bn == 'bn']
prothom = prothom.drop(['lang','lang_bn'],axis=1)

In [12]:
prothom

sentence
0        পশ্চিমবঙ্গের বরখাস্ত হওয়া মন্ত্রী পার্থ চট্টোপ...
1                        আর জামিনের আবেদন করলেন না অর্পিতা
2        আজ শুক্রবার দিনের ইডি হেফাজত শেষে তাঁদের হাজির...
3        সেখানে পার্থর আইনজীবী জামিনের আবেদন জানালেও অর...
4                               অর্পিতা মুখার্জি কারাগারেই
...                                                    ...
8418467        পাঁচটি অভিযোগই ট্রাইব্যুনালে প্রমাণিত হয়েছে
8418468            চারটি অভিযোগে তাঁকে মৃত্যুদণ্ড দেওয়া হয়
8418469  এর মধ্যে রাজনৈতিক গ্রুপকে গণহত্যার দায়ের বিষয়ট...
8418470                                           যা প্রথম
8418471  অপর একটি অভিযোগে তাঁকে বছরের কারাদণ্ড দিয়েছেন ...

[8277516 rows x 1 columns]

In [13]:
freq = {}
for sentence in  tqdm(prothom['sentence']):
    sen = sentence.split()
    for item in sen:
        if (item in freq):
            freq[item] += 1
        else:
            freq[item] = 1

100%|██████████| 8277516/8277516 [00:28<00:00, 292862.83it/s]


In [14]:
df = pd.DataFrame()

In [15]:
words = list(freq.keys())
freq = list(freq.values())
df['word'] = words
df['frequency'] = freq

In [16]:
df = df.dropna(how='any')

In [17]:
df_temp = df.merge(df_99k,how='right',on='word')

df_temp = df_temp[df_temp.isna().any(axis=1)].drop('frequency_x',axis=1)
df_temp.rename(columns = {'frequency_y':'frequency'}, inplace = True)
df_temp

word  frequency
355            মিঃ        839
491          তাইনা        561
505          দাড়াও        546
622        পারবেনা        413
687           ইয়াহ        377
...            ...        ...
83962     ক্রেবাইন          1
83963  কারাধ্রাসএর          1
83965     ফ্রোডোকে          1
83966  কারাধ্রাসের          1
83967   অ্যাডলারকে          1

[33979 rows x 2 columns]

In [18]:
df=pd.concat([df,df_temp])

In [19]:
df

word  frequency
0      পশ্চিমবঙ্গের       2525
1           বরখাস্ত       2021
2              হওয়া      28354
3           মন্ত্রী       6635
4             পার্থ       1351
...             ...        ...
83962      ক্রেবাইন          1
83963   কারাধ্রাসএর          1
83965      ফ্রোডোকে          1
83966   কারাধ্রাসের          1
83967    অ্যাডলারকে          1

[444169 rows x 2 columns]

In [20]:
df.to_csv(r'prothom_alo_word_freq.txt', header=None, index=None, sep=' ', mode='a')

In [21]:
from itertools import islice

import pkg_resources
from symspellpy import SymSpell

sym_spell = SymSpell()

sym_spell.load_dictionary('freq.txt', 0, 1)

# Print out first 5 elements to demonstrate that dictionary is
# successfully loaded
print(list(islice(sym_spell.words.items(), 5)))

2022-12-01 23:01:59,651: E symspellpy.symspellpy] Dictionary file not found at freq.txt.


[]
